## Creating a combined dataframe of NY City and Toronto Boroughs

In [1]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


!conda install -c conda-forge geocoder --yes 


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#Scraping the table from wiki and placing it in a pandas dataframe
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
Borough = df["Borough"].tolist()
df = [Borough]
df = pd.DataFrame(df)
df = df.transpose()
df.rename(columns = {0: "Borough"}, inplace = True)

#Removing all "Not assigned" and then limiting the information to only unique boroughs
df = df[df.Borough != 'Not assigned']
df = df["Borough"].unique()
df = pd.DataFrame(df)
df.rename(columns = {0: "Borough"}, inplace = True)


#For loop to find the latitude and longitude of each borough
latitude = []
longitude = []
for B in df["Borough"]:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(B))
        lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1]) 

B = df["Borough"].tolist()

city = []
for n in df["Borough"]:
    city.append("Toronto")
    
df_latlong = pd.DataFrame([city, B, latitude, longitude])
df_latlong_Toronto = df_latlong.transpose()
df_latlong_Toronto.rename(columns = {0: "City", 1:"Borough", 2: "Latitude", 3: "Longitude"}, inplace = True)

df_latlong_Toronto =  df_latlong_Toronto[df_latlong_Toronto['Borough'].str.contains("Toronto")]

In [3]:
import json # library to handle JSON files


!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    

In [4]:
column_names = ['City','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods_data = newyork_data['features']

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    city = "New York"
    
    neighborhoods = neighborhoods.append({'City' : city, 'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
 
ny_df = neighborhoods.drop(['Neighborhood','Latitude','Longitude'], axis = 1)
ny_df = ny_df["Borough"].unique()

In [5]:

lat = []
long = []
for B in ny_df:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, New York City, NY'.format(B))
        lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    long.append(lat_lng_coords[1]) 



In [6]:
city = []
for n in ny_df:
    city.append("New York")

ny_df_new = ny_df.tolist()
ny_df_new

['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island']

In [7]:
df_latlong_ny = pd.DataFrame([city, ny_df_new, lat, long])
df_latlong_ny = df_latlong_ny.transpose()
df_latlong_ny.rename(columns = {0: "City", 1:"Borough", 2: "Latitude", 3: "Longitude"}, inplace = True)

df_latlong_ny

,City,Borough,Latitude,Longitude
0,New York,Bronx,40.8851,-73.8665
1,New York,Manhattan,40.7146,-74.0071
2,New York,Brooklyn,40.6293,-73.9439
3,New York,Queens,40.7526,-73.9388
4,New York,Staten Island,40.5811,-74.1634


In [8]:
frames = [df_latlong_Toronto, df_latlong_ny]
comb_df = pd.concat(frames)
comb_df

,City,Borough,Latitude,Longitude
1,Toronto,Downtown Toronto,43.6595,-79.3554
7,Toronto,East Toronto,43.6778,-79.5014
8,Toronto,West Toronto,43.6492,-79.468
9,Toronto,Central Toronto,43.6097,-79.4928
0,New York,Bronx,40.8851,-73.8665
1,New York,Manhattan,40.7146,-74.0071
2,New York,Brooklyn,40.6293,-73.9439
3,New York,Queens,40.7526,-73.9388
4,New York,Staten Island,40.5811,-74.1634


## Exporting the venue data from Foursquare

In [9]:
CLIENT_ID = 'HRLJQDSL22E520PIRUKGAWSVOLJ0YUW51QF0STZLQHANL54E' # your Foursquare ID
CLIENT_SECRET = '5ZCV0JXUZWAQYOQDNZ3KVJXZVJSS2VQ1UEUOOSFZYAGOKYLP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 1000
                     
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
All_venues = getNearbyVenues(names=comb_df['Borough'],
                                   latitudes=comb_df['Latitude'],
                                   longitudes=comb_df['Longitude']
                                  )

print(All_venues.shape)
All_venues.head()

Downtown Toronto
East Toronto
West Toronto
Central Toronto
Bronx
Manhattan
Brooklyn
Queens
Staten Island
(358, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65952,-79.35543,Merchants of Green Coffee,43.659916,-79.353963,Coffee Shop
1,Downtown Toronto,43.65952,-79.35543,Dark Horse Espresso Bar,43.658498,-79.352356,Coffee Shop
2,Downtown Toronto,43.65952,-79.35543,White Lily Diner,43.658675,-79.351074,Diner
3,Downtown Toronto,43.65952,-79.35543,Quad East,43.658681,-79.351273,Gym / Fitness Center
4,Downtown Toronto,43.65952,-79.35543,Impact Kitchen,43.656369,-79.356980,Restaurant


In [11]:
All_venues.groupby('Borough').count();
print('There are {} uniques categories.'.format(len(All_venues['Venue Category'].unique())))


There are 140 uniques categories.


## Analysis of venue data

In [12]:
# one hot encoding
All_venues_onehot = pd.get_dummies(All_venues[['Venue Category']], prefix="", prefix_sep="")

# add Borough column back to dataframe
All_venues_onehot['Borough'] = All_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [All_venues_onehot.columns[-1]] + list(All_venues_onehot.columns[:-1])
All_venues_onehot = All_venues_onehot[fixed_columns]
All_venues_onehot

,Borough,Accessories Store,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Australian Restaurant,...,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Downtown Toronto,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
All_venues_grouped = All_venues_onehot.groupby('Borough').mean().reset_index()
All_venues_grouped

,Borough,Accessories Store,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Australian Restaurant,...,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bronx,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
1,Brooklyn,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.018182,0.000000,0.018182,0.00,0.00,0.00,0.00,0.000000,0.00
2,Central Toronto,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
3,Downtown Toronto,0.000000,0.00,0.000000,0.028571,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
4,East Toronto,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.142857,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
5,Manhattan,0.000000,0.03,0.000000,0.000000,0.01,0.000000,0.01,0.010000,0.01,...,0.000000,0.010000,0.000000,0.010000,0.01,0.01,0.02,0.01,0.000000,0.01
6,Queens,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.013699,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00
7,Staten Island,0.021739,0.00,0.000000,0.000000,0.00,0.021739,0.00,0.021739,0.00,...,0.000000,0.000000,0.000000,0.065217,0.00,0.00,0.00,0.00,0.021739,0.00
8,West Toronto,0.000000,0.00,0.071429,0.000000,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.000000,0.071429,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00


In [14]:
num_top_venues = 10

for hood in All_venues_grouped['Borough']:
    print("----"+hood+"----")
    temp = All_venues_grouped[All_venues_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bronx----
                  venue  freq
0           Pizza Place  0.16
1  Caribbean Restaurant  0.16
2                   Spa  0.11
3  Fast Food Restaurant  0.05
4         Moving Target  0.05
5            Restaurant  0.05
6        Discount Store  0.05
7    Seafood Restaurant  0.05
8            Shoe Store  0.05
9            Smoke Shop  0.05


----Brooklyn----
                  venue  freq
0     Mobile Phone Shop  0.07
1   Sporting Goods Shop  0.05
2            Donut Shop  0.04
3              Pharmacy  0.04
4        Shipping Store  0.04
5           Pizza Place  0.04
6  Caribbean Restaurant  0.04
7    Chinese Restaurant  0.04
8           Supermarket  0.04
9                  Bank  0.04


----Central Toronto----
                             venue  freq
0                             Park  0.33
1                Convenience Store  0.22
2                   Breakfast Spot  0.11
3                    Grocery Store  0.11
4                             Bank  0.11
5                     Tennis Court 

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = All_venues_grouped['Borough']

for ind in np.arange(All_venues_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(All_venues_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Caribbean Restaurant,Pizza Place,Spa,Discount Store,Moving Target,Cosmetics Shop,Restaurant,Seafood Restaurant,Shoe Store,Smoke Shop
1,Brooklyn,Mobile Phone Shop,Sporting Goods Shop,Pizza Place,Bank,Caribbean Restaurant,Supermarket,Shipping Store,Chinese Restaurant,Pharmacy,Donut Shop
2,Central Toronto,Park,Convenience Store,Tennis Court,Grocery Store,Bank,Breakfast Spot,Curling Ice,Dessert Shop,Department Store,Deli / Bodega
3,Downtown Toronto,Coffee Shop,Restaurant,Pizza Place,Park,Convenience Store,Pub,Pool,Café,Gastropub,Health Food Store
4,East Toronto,Park,Coffee Shop,Hockey Arena,Construction & Landscaping,Asian Restaurant,Restaurant,Cuban Restaurant,Dessert Shop,Department Store,Deli / Bodega
5,Manhattan,Coffee Shop,Italian Restaurant,Hotel,Sandwich Place,French Restaurant,Gym / Fitness Center,Spa,Gym,Cocktail Bar,Park
6,Queens,Hotel,Coffee Shop,Deli / Bodega,Café,Bar,Donut Shop,Mexican Restaurant,Sandwich Place,Rental Car Location,Pizza Place
7,Staten Island,Clothing Store,Toy / Game Store,Cosmetics Shop,Department Store,Accessories Store,Bus Stop,Coffee Shop,Moving Target,Movie Theater,Mobile Phone Shop
8,West Toronto,Pharmacy,Pizza Place,Mexican Restaurant,Soccer Field,Baseball Field,Dog Run,Pub,Café,Pool,Theme Park Ride / Attraction


## K-means Clustering

In [16]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

All_venues_grouped_clustering = All_venues_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(All_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 3, 1, 2, 1, 1, 1, 0], dtype=int32)

In [17]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
comb_df_merged = comb_df.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

comb_df_merged # check the last columns!

,City,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Toronto,Downtown Toronto,43.6595,-79.3554,1,Coffee Shop,Restaurant,Pizza Place,Park,Convenience Store,Pub,Pool,Café,Gastropub,Health Food Store
7,Toronto,East Toronto,43.6778,-79.5014,2,Park,Coffee Shop,Hockey Arena,Construction & Landscaping,Asian Restaurant,Restaurant,Cuban Restaurant,Dessert Shop,Department Store,Deli / Bodega
8,Toronto,West Toronto,43.6492,-79.468,0,Pharmacy,Pizza Place,Mexican Restaurant,Soccer Field,Baseball Field,Dog Run,Pub,Café,Pool,Theme Park Ride / Attraction
9,Toronto,Central Toronto,43.6097,-79.4928,3,Park,Convenience Store,Tennis Court,Grocery Store,Bank,Breakfast Spot,Curling Ice,Dessert Shop,Department Store,Deli / Bodega
0,New York,Bronx,40.8851,-73.8665,4,Caribbean Restaurant,Pizza Place,Spa,Discount Store,Moving Target,Cosmetics Shop,Restaurant,Seafood Restaurant,Shoe Store,Smoke Shop
1,New York,Manhattan,40.7146,-74.0071,1,Coffee Shop,Italian Restaurant,Hotel,Sandwich Place,French Restaurant,Gym / Fitness Center,Spa,Gym,Cocktail Bar,Park
2,New York,Brooklyn,40.6293,-73.9439,1,Mobile Phone Shop,Sporting Goods Shop,Pizza Place,Bank,Caribbean Restaurant,Supermarket,Shipping Store,Chinese Restaurant,Pharmacy,Donut Shop
3,New York,Queens,40.7526,-73.9388,1,Hotel,Coffee Shop,Deli / Bodega,Café,Bar,Donut Shop,Mexican Restaurant,Sandwich Place,Rental Car Location,Pizza Place
4,New York,Staten Island,40.5811,-74.1634,1,Clothing Store,Toy / Game Store,Cosmetics Shop,Department Store,Accessories Store,Bus Stop,Coffee Shop,Moving Target,Movie Theater,Mobile Phone Shop


In [18]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
tor_latitude = location.latitude
tor_longitude = location.longitude

In [19]:
map_clusters = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(comb_df_merged['Latitude'], comb_df_merged['Longitude'], comb_df_merged['Borough'], comb_df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [20]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="NY_explorer")
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude

In [21]:
map_clusters = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(comb_df_merged['Latitude'], comb_df_merged['Longitude'], comb_df_merged['Borough'], comb_df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters